# Práctica 2: Técnicas de análisis
## Minería de Datos, Máster Universitario en Ciencia de Datos
## Jorge García Carrasco

In [ ]:
!pip install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

# para que pandas muestre todas las columnas del DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 67)

import datetime

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

# Introducción

El objetivo de esta práctica consiste en familiarizarse con las diferentes técnicas de análisis propias de la minería de datos. 
Partiremos del dataset **EM-DAT** (https://www.emdat.be/), que contiene información sobre los diferentes desastres que han ocurrido a lo largo del planeta, desde 1900 hasta la actualidad.

Preprocesaremos y echaremos un primer vistazo al dataset, para familiarizarnos con las variables presentes, observar si faltan valores...

A continuación, comenzaremos con la **Parte 1** de la práctica, en la que se utilizarán diversos algoritmos para un problema de **regresión** (es decir, intentar buscar la relación entre las variables explicativas y una variable respuesta numérica), y un problema de **clasificación** (en este caso, la variable respuesta es **categórica**). Una vez se hayan probado varios algoritmos se evaluarán y se analizarán los resultados que se hayan podido extraer. 

Concretamente, el problema de regresión consistirá en predecir los costes ocasionados por el desastre, mientras que en el problema de clasificación se tratará de clasificar los desastres en diferentes grupos según su naturaleza.

Finalmente, para la **Parte 2**, realizaremos una tarea de **ensembles** y otra de **clustering**. Para la tarea de ensembles se utilizará el mismo dataset que en la parte 1, pero para la parte de clustering se utilizarán datos sobre las estadísticas de los campeones de League of Legends.

# Primer vistazo al dataset

Los siguientes enlaces contienen una gran cantidad de información sobre el dataset:
- https://emdat.be/guidelines
-http://drm.cenn.org/Trainings/Multi%20Hazard%20Risk%20Assessment/Lectures_ENG/Session%2001%20Introduction%20to%20risk%20management/Tasks/disaster%20database/EM%20dat%20data%20base%20guidelines.doc

Vamos a echar un primer vistazo:

In [ ]:
emdat = pd.read_excel('../input/emdat/emdat.xlsx')
emdat.tail()

Tal y como está especificado en la página, en el dataset de EM-DAT se consideran como desastres aquellos que cumplan al menos uno de estos criterios:

- $10 \geq$ muertos.
- $100 \geq$ personas afectadas.
- Se ha declarado estado de alarma debido a este evento.
- Se ha pedido ayuda internacional

El criterio escogido para incluir el desastre en el dataset se indica en la columna `Entry Criteria`:

In [ ]:
emdat['Entry Criteria'].value_counts().plot(kind='bar')

En en enlace (http://drm.cenn.org/Trainings/Multi%20Hazard%20Risk%20Assessment/Lectures_ENG/Session%2001%20Introduction%20to%20risk%20management/Tasks/disaster%20database/EM%20dat%20data%20base%20guidelines.doc) explican algunos de los códigos utilizados:

- **Kill**: 10 or more people killed
- **Affected**: 100 or more people affected/injured/homeless
- **SigDis**: Significant disaster (e.g. «second worst »)
- **SigDam**: Significant damage
- **Decla/int**: Declaration of a state of emergency or/and appeal for an international assistance
- **Regional**: Disaster entered at the country level without data, because it has affected several countries/regions.
- **Unknown**: Reason not known (old entries)

Siguiendo estas pautas, juntaremos varias categorías (por ejemplo, `Affected` y `Affect`, `Declar` y `Declar/Int`), y todas las categorías que no aparezcan en el listado de arriba las clasificaremos como `Unknown`:

In [ ]:
emdat['Entry Criteria'] = emdat['Entry Criteria'].replace("Affect", "Affected")
emdat['Entry Criteria'] = emdat['Entry Criteria'].replace("Declar", "Declar/Int")
emdat['Entry Criteria'] = emdat['Entry Criteria'].replace(["Waiting", "OFDA", "Govern", "--"], "Unknown")

In [ ]:
emdat['Entry Criteria'].value_counts().plot(kind='bar')

In [ ]:
n = len(emdat)
print('El dataset contiene {} desastres'.format(n))

In [ ]:
emdat['Disaster Group'].value_counts()

El dataset esta compuesto por un total de 24923 desastres, clasificados en dos grandes grupos: desastres de carácter **natural**, y desastres de carácter **tecnológico**. También hay un tercer grupo muy reducido denominado **Complex disasters**, o desastres complejos, compuesto por sólo 14 desastres.

A su vez, estos grupos se dividen en subgrupos, y una gran variedad de tipos y subtipos:

In [ ]:
df = emdat[['Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype']].groupby(['Disaster Group', 'Disaster Subgroup', 'Disaster Type'])['Disaster Subtype'].unique().to_frame()
df.explode('Disaster Subtype').dropna()

Como podemos observar, la cantidad de tipos de desastre es abrumadora. Estos tipos y subtipos están descritos en la página de EM-DAT: https://www.emdat.be/classification.

También se puede observar que el grupo `Complex Disasters` sólo contiene desastres del subtipo `Famine`, es decir, hambruna.

Ahora, vamos a tratar los datos de fecha de inicio y final del desastre. Primero, debemos decidir qué hacer con los valores ausentes:

In [ ]:
emdat[['Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day']].isna().sum(axis=0)/n

Podemos ver que los meses de inicio y final del desastre no están específicados para alrededor de un 2% y 3% respectivamente. Podríamos tratar de aplicar técnicas de imputación, pero para un porcentaje tan bajo no vale la pena. Además, los diferentes desastres son independientes entre sí, y no tendría sentido sustituir los valores ausentes por la moda.

Es probable que diferentes tipos de desastres ocurran en una temporada determinada (por ejemplo, las sequías suelen pasar en meses de verano, y las heladas en meses de invierno). Tal vez, esta podría ser una buena estrategia a estudiar, pero por ahora simplemente **eliminaremos** los desastres en los que se desconozca el mes de inicio y final.

In [ ]:
emdat = emdat[emdat['Start Month'].notna() & emdat['End Month'].notna()]

print('Se han eliminado {} columnas ({:.2f}% de los datos)'.format(n - len(emdat), (n - len(emdat))/n*100))
# actualizamos el numero de muestras del dataset
n = len(emdat)

El día de inicio y final del desastre no está especificado para aproximadamente un 15% de los datos. Sin embargo, el día de inicio y final no parece que sea un valor relevante a la hora de analizar los datos, por tanto, reemplazaremos los valores ausentes simplemente por el **primer** día del mes:

In [ ]:
emdat[['Start Day', 'End Day']] = emdat[['Start Day', 'End Day']].fillna(1)

Una vez hemos tratado con los valores ausentes, podemos convertir los datos a formato `datetime`:

In [ ]:
# Tomamos las fechas de inicio
df = emdat[['Start Year', 'Start Month', 'Start Day']].astype('int32')
# cambiamos el nombre de las columnas
df = df.rename(columns={'Start Year': 'year',
                   'Start Month': 'month',
                   'Start Day': 'day'})
df = pd.to_datetime(df, errors='coerce')
emdat[df.isna()]

Vemos que hay una fecha que da error a la hora de convertir a formato `datetime`. Analizando con mayor detalle, vemos que el error salta porque esa fecha **no existe**: el inicio del desastre se ha registrado como el día 31 de septiembre de 1992, pero el mes de septiembre de 1992 sólo tenia 30 días. 

Como sólo se trata de un error, lo eliminaremos del dataset.

In [ ]:
emdat = emdat[df.notna()]
df = df[df.notna()]
# Creamos la columna con la fecha de inicio del desastre en un formato correcto
emdat['Start Date'] = df
# Eliminamos las columnas de año, mes y día
#emdat = emdat.drop(['Start Year', 'Start Month', 'Start Day'], axis=1)

Realizamos el mismo proceso para las fechas de fin del desastre:

In [ ]:
# Tomamos las fechas de inicio
df = emdat[['End Year', 'End Month', 'End Day']].astype('int32')
# cambiamos el nombre de las columnas
df = df.rename(columns={'End Year': 'year',
                   'End Month': 'month',
                   'End Day': 'day'})
df = pd.to_datetime(df, errors='coerce')
emdat[df.isna()]

In [ ]:
emdat = emdat[df.notna()]
df = df[df.notna()]
# Creamos la columna con la fecha de inicio del desastre en un formato correcto
emdat['End Date'] = df
# Eliminamos las columnas de año, mes y día
#emdat = emdat.drop(['End Year', 'End Month', 'End Day'], axis=1)

Una vez tenemos las fechas en el formato adecuado, las podemos utilizar para crear la variable `Duration`, compuesta por la **duración** del desastre en días. Esta variable puede resultar útil a la hora de buscar patrones y aplicar algoritmos:

In [ ]:
emdat['Duration'] = emdat['End Date'] - emdat['Start Date'] + datetime.timedelta(days=1)
emdat['Duration'] = emdat['Duration'].dt.days
emdat.sort_values(by='Duration')

In [ ]:
len(emdat[(emdat['Duration'] < 0)])

En el código anterior podemos ver que hay desastres con **duración negativa**. Evidentemente esto se trata de un error; puede que al introducir el desastre en la base de datos se hayan confundido entre fecha de inicio y fecha final. 

Podríamos tratar de revertir las fechas, pero como sólo se tratan de 14 muestras, las eliminaremos.

In [ ]:
emdat = emdat[emdat['Duration']>0]

# Parte 1: Regresión y Clasificación
## Regresión

Antes de comenzar, es importante recalcar que el objetivo de los próximos algoritmos **no es el de predecir**, ya que el momento en el que ocurre un desastre y sus consecuencias tiene un fuerte carácter aleatorio. Por tanto, el objetivo de estos algoritmos consistirá en **analizar** la relación y patrones entre las diferentes variables, en otras palabras, se utilizarán como herramienta para la **extracción de conocimiento** a partir de los datos.

La variable que analizaremos será la de los daños estimados causados por el desastre, `Total Damages`. En la página de EM-DAT, se indica que esta variable representa el daño estimado a la propiedad, cultivos y ganado, en miles de dólares (\$). Es importante recalcar que el **valor es exactamente el mismo que se estimó en el propio año del desastre** y no tiene en cuenta otras variables como la inflación.

Como podemos apreciar, este valor no está indicado para **aproximadamente un 78\% de los desastres**. Desgraciadamente, no podremos utilizar esas muestras para el análisis, y deberemos descartarlos. Sin embargo, el dataset es bastante grande y nos quedan un total de 5302 muestras, que serán suficientes para nuestro análisis de regresión.

In [ ]:
n = len(emdat)
(emdat.isna().sum(axis=0)/n).sort_values().to_frame('% de valores ausentes')

In [ ]:
print('El valor `Total Damages` está indicado para {} muestras'.format(len(emdat['Total Damages (\'000 US$)'].dropna())))

In [ ]:
df_reg = emdat[emdat['Total Damages (\'000 US$)'].notna()]
# añadimos la columna de Start Month
df_reg['Start Month'] = df_reg['Start Date'].dt.month
X, y = df_reg.drop(['Total Damages (\'000 US$)'], axis=1), df_reg['Total Damages (\'000 US$)']

In [ ]:
n = len(df_reg)
(df_reg.isna().sum(axis=0)/n).sort_values().to_frame('% de valores ausentes')

Para la tarea de regresión, utilizaremos las siguientes variables numéricas:

- `Total Deaths`: Es esperable que a mayor número de muertes, mayor sean los costes del desastre.

- `Total Affected`: De nuevo, se espera que a mayor número de afectados, mayor será el desastre y su coste asociado.

- `Duration`: También se espera que la duración tenga alguna relación con el coste.

- `Year`:  Probablemente el año tenga relación con el coste debido a la inflación.

Respecto a las variables categóricas, utilizaremos:

- `Continent`: La incluimos para estudiar si hay cierta dependencia del coste con la localización del desastre.

- `Entry Criteria`: Ciertos criterios de entrada, como la declaración del estado de alarma, dan información sobre la magnitud del desastre, y por tanto pueden estar relacionados con el coste.

- `Start Month`: Se incluye para ver si hay cierta dependencia estacional con el coste o calibre del desastre.

- `Disaster Type`, `Disaster Group` y `Disaster Subgroup`: Se espera que ciertos tipos de desastres supongan más coste que otros (por ejemplo, un tsunami o terremoto puede ser mucho más catastrófico que un incendio)


Crearemos un pipeline de preprocesado de datos que conectaremos a los diferentes algoritmos. Inicialmente, se tendrán dos pipelines diferentes, una para las variables numéricas y otro para las categóricas:

- El pipeline de las variables numéricas estará formado por un `SimpleImputer`, que simplemente rellenará valores ausentes con la mediana (en el caso de que hubieran casos ausentes), y un `StandardScaler`, que escala las variables para que formen una distribución de media cero y desviación estándar 1. Estandarizar es muy importante para que funcionen correctamente ciertos algoritmos como kNN o SVR.

- El pipeline de las variables categóricas estarán formado simplemente por un `OneHotEncoder`

Estos dos pipelines se unirán y se le conectará el algoritmo deseado.

Probaremos primero con una simple **regresión lineal** para tener un modelo base de referencia. **Fijaremos una semilla** para poder reproducir los resultados, aunque es importante recalcar que en una **situación real** esto no se debe hacer, ya que algunos resultados pueden variar de una ejecución a otra.

In [ ]:
np.random.seed(42)

In [ ]:
# Lista de variables numericas que vamos a utilizar
numeric_features = ['Total Deaths', 
                    'Total Affected', 
                    'Duration', 
                    'Year']
# Lista de variables categóricas
categorical_features = ['Continent', 
                        'Entry Criteria', 
                        'Start Month',
                        'Disaster Type',
                        'Disaster Group',
                        'Disaster Subgroup']

# Pipeline de procesado de variables numericas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Pipeline de procesado de variables categoricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('linear_reg', LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0)

reg.fit(X_train, y_train)
print("R^2: %.3f" % reg.score(X_test, y_test))

In [ ]:
from sklearn.model_selection import cross_validate

def summary(model, X, y, cv=5, scoring=('r2', 'neg_root_mean_squared_error')):
    """ Función auxiliar que evalúa un modelo utilizando cross-validation y
        imprime un pequeño resumen.
    """
    cv_result = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)
    scores = -cv_result['test_neg_root_mean_squared_error']
    rmse = scores.mean()
    rmse_train = -cv_result['train_neg_root_mean_squared_error'].mean()
    print("RESUMEN CV=5, modelo {}".format(model.steps[-1][0]))
    print("RMSE_Train: {:.0f}".format(rmse_train))
    print("RMSE_Test: {:.0f}".format(rmse))
    print("Valor medio de la variable Total damages: {:.0f}".format(y.mean()))
    print("RMSE_Test/(Valor medio): {:.03f}%".format(rmse/y.mean()*100))
    print("R2: {}".format(cv_result['test_r2'].mean()))

In [ ]:
summary(reg, X, y)

Evaluando el modelo con cross-validation (con 5 folds), obtenemos un RMSE **8 veces mayor** que la media de la variable que intentamos predecir. Se considera que el ajuste es bueno cuando el RMSE es aproximadamente un 15\% del valor medio. Además, se obtiene un $R^2$ negativo, indicando que el modelo no es capaz de explicar la relación entre las variables explicativas y la variable respuesta.

Los resultados desde luego no son prometedores, pero hemos de tener en cuenta que un modelo de regresión lineal sólo puede captar dependencias lineales, y es probable que las variables que hemos escogido tengan no-linealidades. Por tanto, vamos a probar modelos que puedan captar patrones no lineales, como pueden ser los algoritmos de **KNN**, **Decision Trees** y los **SVM** con kernel no lineal.



In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = Pipeline(steps=[('preprocessor', preprocessor),
                      ('knn', KNeighborsRegressor())])

summary(knn, X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtree = Pipeline(steps=[('preprocessor', preprocessor),
                      ('dtree', DecisionTreeRegressor(max_depth=1, max_features=5))])

summary(dtree, X, y)

In [ ]:
from sklearn.svm import SVR

svr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('svr', SVR())])

summary(svr, X, y)

Al usar modelos no lineales el RMSE se reduce considerablemente, y el $R^2$ ya no tiene un valor tan negativo. El mejor modelo parece ser el SVR, con un $R^2$ cercano a 0. Evidentemente, un $R^2$ próximo a 0 sigue siendo un mal resultado, pero es una mejoría. 

Ahora, intentaremos hacer un ajuste de hiperparámetros para ver si podemos mejorar el modelo ligeramente: cambiando el kernel, o sus parámetros `C` y `epsilon`.

Para realizar la búsqueda utilizaremos `RandomizedSearchCV`, de sklearn. Esta implementación nos permite automatizar relativamente la búsqueda de hiperparámetros: indicamos en un diccionario las distribuciones de las que queremos extraer los hiperparámetros, y `RandomizedSearchCV` va extrayendo hiperparámetros y evaluandolos por CV. 

Los parámetros se extraerán de las siguientes distribuciones:

- `C` y `epsilon` se extraerán de una distribución uniforme [0, 4].
- El kernel se escogerá aleatoriamente entre ['rbf', 'sigmoid' y 'poly']

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# distribuciones de las que se extraerán los hiperparámetros
distributions = {'svr__C':uniform(loc=0, scale=4),
                 'svr__epsilon':uniform(loc=0, scale=4),
                 'svr__kernel':['rbf', 'sigmoid', 'poly']}
svr_hyper = RandomizedSearchCV(svr, distributions)
# iniciamos la busqueda de hiperparametros
search = svr_hyper.fit(X, y)
# Mejor conjunto de hiperparametros
search.best_params_

Después de completar la búsqueda, podemos visualizar los mejores hiperpámetros y acceder al modelo final con dichos hiperparámetros:

In [ ]:
best_svr = search.best_estimator_

summary(best_svr, X, y)

Como vemos, no hay prácticamente mejora. 

A partir de estos resultados podemos afirmar casi con seguridad que el problema que intentamos resolver no tiene solución, ya que es demasiado aleatorio. 

Inicialmente, se pensaba que según el tipo de desastre, cuando ocurrió, su duración... se conseguiría obtener algún tipo de relación con los daños totales. Sin embargo, parece que es demasiado aleatorio o no hay información suficiente como para extraer conclusiones.

Como último intento, probaremos a corregir el coste de los daños respecto a la inflación.

En la página de EM-DAT mencionan que los precios que aparecen en `Total Damages` son los que se reportaron en la época, sin tener en cuenta la **inflación**. Sin embargo, en el dataset hay una columna `CPI`, que contiene el denominado **Consumer Price Index**. El CPI representa un ratio estimado entre el dinero que cuesta una cesta de la compra en un determinado momento comparado con un año anterior de referencia (https://www.investopedia.com/terms/c/consumerpriceindex.asp). Por ejemplo, si el CPI en el año 2000 es de 10, significa que pagamos 10 veces más por el mismo producto que en un año de referencia anterior (no es necesario saber el año de referencia en nuestro caso)

In [ ]:
sns.barplot(data=emdat, x='Year', y='Total Damages (\'000 US$)')

Podemos ver que, a medida que avanzan los años, los costes representados en dólares aumenta debido a la inflación. Esto impide que nosotros seamos capaces de comparar los daños **reales** de desastres que hayan ocurrido en epocas distintas.

Por ejemplo, puede ocurrir que hayan ocurrido dos desastres similares en los años 1920, y en 2000, pero debido a la inflación, el coste representado en dólares sea mucho mayor en el desastre del 2000.

En resumen, al corregir la inflación, estudiaremos una medida del **coste real** del desastre.

Probamos a corregir la inflación:

In [ ]:
emdat['Total Damages Corregida'] = emdat['Total Damages (\'000 US$)']/emdat['CPI']

In [ ]:
sns.barplot(data=emdat, x='Year', y='Total Damages Corregida')

Parece que los costes una vez corregidos tienen más sentido, ya que podemos comparar desastres de diferentes épocas.

Probaremos ahora varios modelos y analizaremos los resultados:

In [ ]:
df_reg = emdat[emdat['Total Damages Corregida'].notna()]
# añadimos la columna de Start Month
df_reg['Start Month'] = df_reg['Start Date'].dt.month
X, y = df_reg.drop(['Total Damages Corregida'], axis=1), df_reg['Total Damages Corregida']

In [ ]:
summary(reg, X, y)

In [ ]:
summary(knn, X, y)

In [ ]:
summary(dtree, X, y)

In [ ]:
summary(svr, X, y)

Notamos una mejoría en el modelo lineal. Esto tiene sentido, ya que de alguna manera, hemos modificado la variable respuesta para que se pueda ajustar mejor mediante una recta.

Sin embargo, los resultados de los demás modelos siguen siendo poco prometedores.

## Clasificación

Ahora, estudiaremos si existe alguna relación entre el subgrupo de desastre, `Disaster Subgroup`, y el resto de variables. Escogemos el subgrupo y no el grupo, porque de los 3 grupos presentes (Natural, Technological y Complex Disasters) hay muy pocas observaciones de este último. 

In [ ]:
emdat['Disaster Group'].value_counts().plot(kind='bar')

Intentaremos predecir la variable `Disaster Subgroup`. Hay un total de 8 clases:

In [ ]:
emdat['Disaster Subgroup'].value_counts().plot(kind='bar')

Se puede ver que hay un desbalance de clases, es decir, hay más muestras de unas clases que de otras. Esto puede provocar ciertos problemas, sobretodo a la hora de evaluar el modelo. En estos casos, la accuracy no es una buena métrica, así que nos decantaremos por el **F1 score**. En el caso multiclase, hay varias formas de calcular el F1, en concreto, **micro** o **macro** average. 

La elección de la métrica depende es una decisión que hemos de tomar. Generalmente, el micro average se suele escoger cuando se quiere maximizar el número de aciertos, sin tener en cuenta las clases. Sin embargo, el macro average tiene en cuenta la precisión de aciertos en cada clase. 

Por poner un ejemplo, si se quiere predecir una enfermedad que posee el 1% de la población, un clasificador que siempre predice que el individuo está sano, tendra un **F1 score calculado con micro average de 0.99**, cuando evidentemente, no es un buen clasificador para la tarea. En conclusión, como en el problema actual se desea que el clasificador funcione correctamente con todas las clases posibles, utilizaremos el F1 score calculado con **macro average**. (Para más información sobre el F1 score, acudir a la discusión https://datascience.stackexchange.com/questions/36862/macro-or-micro-average-for-imbalanced-class-problems).

Evidentemente, no podemos escoger las variables 'Disaster Group', 'Disaster Type', 'Disaster Subtype' y 'Disaster Subsubtype' como variables respuesta porque tienen relación directa con la variable respuesta, así que las descartamos

In [ ]:
# Lista de variables numericas que vamos a utilizar
numeric_features = ['Total Deaths', 
                    'Total Affected', 
                    'Duration',
                    'Total Damages Corregida',
                    'Year']
# Lista de variables categóricas
categorical_features = ['Continent', 
                        'Entry Criteria', 
                        'Start Month']


In [ ]:
X, y = emdat[numeric_features+categorical_features], emdat['Disaster Subgroup']

In [ ]:
X.isna().sum(axis=0)/len(X)

Vemos que la variable `Total Damages Corregida` está ausente para un 80% de los valores. La cantidad de valores ausentes es demasiado alta como para utilizar técnicas de imputación, así que eliminaremos aquellos desastres que no contengan dicho valor.

In [ ]:
mask = X['Total Damages Corregida'].notna()
X, y = X[mask], y[mask]
print("El conjunto de entrenamiento contiene {} muestras".format(len(mask)))

In [ ]:
X.isna().sum(axis=0)/len(X)

In [ ]:
y.value_counts().plot(kind='bar')

Podemos ver que después de la limpieza de datos, tan solo hay 6 desastres de tipo `Biological`, y tan sólo uno de tipo `Extra-terrestrial`. Se tratan de desastres muy poco comunes, así que los eliminaremos, ya que trabajar con dos clases tan desbalanceadas respecto al resto es prácticamente imposible, se necesitarían más muestras.

In [ ]:
mask = (y != 'Extra-terrestrial') & (y != 'Biological')
# descartamos las clases extra-terrestrial y biological
X, y = X[mask], y[mask]

In [ ]:
y.value_counts().plot(kind='bar')

Finalmente, nuestro problema a resolver será una clasificación con **5 clases**: `Meteorological`, `Hydrological`, `Geophysical`, `Technological` y `Climatological`

Utilizaremos como **baseline** el modelo `DummyClassifier`. La estrategia de este modelo para predecir consiste en elegir una clase **aleatoriamente** para cada muestra. Este no es un modelo real, pero sirve para tener un F1 score de referencia:

In [ ]:
from sklearn.dummy import DummyClassifier

# Pipeline de procesado de variables numericas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Pipeline de procesado de variables categoricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


baseline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('dummy', DummyClassifier(strategy='uniform'))])
cv_result = cross_validate(baseline, X, y, cv=5, scoring=('f1_macro'), return_train_score=True)
cv_result['test_score'].mean()

Tal y como se esperaba, obtenemos un F1 score de aproximadamente $1/5=0.2$.

Ahora entrenaremos un modelo sencillo, como el modelo de regresión logística, y evaluaremos su F1 respecto al baseline:

In [ ]:
def summary_clf(model, X, y):
    """ Función auxiliar para evaluar distintos clasificadores """
    cv_result = cross_validate(model, X, y, cv=5, scoring=('f1_macro'), return_train_score=True)
    print("=== MODELO {} ===".format(model.steps[-1][0]))
    print("F1 Train: {:.02f}".format(cv_result['train_score'].mean()))
    print("F1 Test: {:.02f}".format(cv_result['test_score'].mean()))

In [ ]:
from sklearn.linear_model import LogisticRegression

log = Pipeline(steps=[('preprocessor', preprocessor),
                      ('log_clf', LogisticRegression(max_iter=1000))])
summary_clf(log, X, y)

Como se puede ver, el modelo de regresión logística supone una mejoría considerable respecto al modelo baseline aleatorio. En un primer intento, apareció la advertencia `ConvergenceWarning: lbfgs failed to converge`, que se solucionó aumentando el número de iteraciones del algoritmo con `max_iter=1000`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('knn_clf', KNeighborsClassifier())])
summary_clf(knn_clf, X, y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('dtree_clf', DecisionTreeClassifier())])
summary_clf(dtree_clf, X, y)

In [ ]:
from sklearn.svm import SVC

svc_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('svc_clf', SVC())])
summary_clf(svc_clf, X, y)

En dos de los tres modelos probados (`KNeighborsClassifier` y `DecisionTreeClassifier`), podemos ver claramente que se están **sobreajustando** a los datos (overfit): el F1 en el conjunto de train es muy alto (el árbol de decisión tiene un F1=1), y el F1 en el conjunto de test es incluso peor que la regresión logística. En estos casos es necesario modificar los hiperparámetros por defecto para aumentar la regularización. Probaremos a modificar el `DecisionTreeClassifier` porque es el que menos tarda, y da resultados parecidos.

Respecto al `SVC`, el resultado es parecido al de la regresión logística, a diferencia de que tarda mucho más tiempo en calcular. 

En conclusión, por ahora vamos a intentar ajustar los hiperparámetros del `DecisionTreeClassifier`.

En la guía de usuario de sklearn, encontramos el siguiente fragmento:

*""Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem. ""*

En resumen, hemos de modificar `max_depth`, `min_samples_split` y `min_samples_leaf` para intentar reducir el error de generalización.

In [ ]:
# distribuciones de las que se extraerán los hiperparámetros
distributions = {'dtree_clf__max_depth':[2, 3, 4, 5, 6 ,7],
                 'dtree_clf__min_samples_split':[2, 3, 4, 5, 6, 7, 8],
                 'dtree_clf__min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8]
                }
dtree_hyper = RandomizedSearchCV(dtree_clf, distributions, random_state=42)
# iniciamos la busqueda de hiperparametros
search = dtree_hyper.fit(X, y)
# Mejor conjunto de hiperparametros
search.best_params_

In [ ]:
summary_clf(search.best_estimator_, X, y)

Una vez hemos ajustado los hiperparámetros del árbol de decisión, obtenemos un F1 por validación cruzada ligeramente superior al obtenido con regresión logística. 

Estudiaremos este modelo en más profundidad, calculando la matriz de confusión:

In [ ]:
from sklearn.metrics import plot_confusion_matrix

# hacemos un split en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Ajustamos el modelo
best_dtree = search.best_estimator_
best_dtree.fit(X_train, y_train)

# predecimos las clases
y_pred = best_dtree.predict(X_test)

# calculamos la matriz de confusión
plot_confusion_matrix(best_dtree, X_test, y_test, xticks_rotation=90, normalize='true')
plt.show()

En la matriz de confusión anterior podemos observar varios aspectos interesantes:

- Lo que más llama la atención, es que no hay ninguna muestra que haya sido predecida con la clase `Technological`. Todas los desastres tecnológicos han sido clasificados como meteorológicos. Una posible explicación a esto es que, mientras que el tipo de desastres naturales puede tener relación con la época del año, la duración... Un desastre tecnológico es más aleatorio respecto a estas variables, y por tanto el modelo es incapaz de predecirlo correctamente.

- La clase `Geophysical` se confunde bastante con la clase `Meteorological`. Puede que su relación con las variables explicativas sea similar, y como la clase meteorológica tiene un mayor número de muestras, esta ''absorbe'' a la clase más pequeña.

- Un tercio de las muestras pertenecientes a la clase `Climatological`, mientras que otro tercio se clasifica erróneamente como `Hydrological`, y otro tercio como `Meteorological`. De nuevo, puede que estas clases tengan una estrecha relación.

Finalmente, sklearn nos permite obtener la importancia de las diferentes variables a la hora de ajustar el modelo:

In [ ]:
best_dtree.steps[-1][1].feature_importances_

Obtenemos una lista de 30 elementos: los 5 primeros corresponden a las variables numéricas, los 6 siguientes corresponden a los contintentes, los 7 siguientes corresponden al `Entry criteria`, y los 12 últimos a los meses del año. Las variables más importantes son:

1. `Duration`: 0.82
2. `Total Deaths:` 0.07
3. `Entry Criteria` = : 0.06
4. `Total Damages Corregida`: 0.05

Se puede apreciar claramente que **la duración del desastre está fuertemente relacionada con el tipo de desastre**. El número de muertos, el criterio de entrada, y los daños totales también parecen estar relacionadas, pero en mucha menor medida.

### Datos desbalanceados

Finalmente, trataremos de aplicar una técnica para tratar con los datos desbalanceados e intentar mejorar el modelo.

Hay varias estrategias para tratar con datos desbalanceados, como el submuestreo de datos para que cada clase tenga el mismo número de muestras, sobremuestrear aplicando *bootstrap*, generar muestras sintéticas... (https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18)

En este caso, `sklearn` implementa una funcionalidad que puede mejorar considerablemente el modelo. Cuando se crea un modelo, el argumento `class_weight='balanced'` asigna automáticamente un **peso a cada muestra en función del número de muestras de cada clase**. En otras palabras, a una muestra que pertenezca a una clase con muchas otras muestras se le asignará un peso menor en la función de coste, mientras que a las muestras de una clase más pequeña se le asignará un peso mayor. De esta manera, se incita al modelo a **no ignorar** las clases menos representadas.

Probaremos esta estrategia con el modelo de regresión logística y el árbol de decisión:

In [ ]:
log = Pipeline(steps=[('preprocessor', preprocessor),
                      ('log_clf', LogisticRegression(class_weight='balanced', max_iter=5000))])
summary_clf(log, X, y)

In [ ]:
dtree_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('dtree_clf', DecisionTreeClassifier(class_weight='balanced'))])
summary_clf(dtree_clf, X, y)

In [ ]:
# distribuciones de las que se extraerán los hiperparámetros
distributions = {'dtree_clf__max_depth':[2, 3, 4, 5, 6 ,7],
                 'dtree_clf__min_samples_split':[2, 3, 4, 5, 6, 7, 8],
                 'dtree_clf__min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8]
                }
dtree_hyper = RandomizedSearchCV(dtree_clf, distributions, random_state=42)
# iniciamos la busqueda de hiperparametros
search = dtree_hyper.fit(X, y)
# Mejor conjunto de hiperparametros
search.best_params_

In [ ]:
summary_clf(search.best_estimator_, X, y)

En el modelo de regresión logística no parece haber una mejoría notable, mientras que en el árbol de decisión, el F1 score ha aumentado de 0.31 a 0.34. 

Estudiemos más en profundidad el modelo calculando la matriz de confusión:

In [ ]:
# hacemos un split en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Ajustamos el modelo
best_dtree = search.best_estimator_
best_dtree.fit(X_train, y_train)

# predecimos las clases
y_pred = best_dtree.predict(X_test)

# calculamos la matriz de confusión
plot_confusion_matrix(best_dtree, X_test, y_test, xticks_rotation=90, normalize='true')
plt.show()

A primera vista, se pueden extraer las siguientes conclusiones:

- La clase `Technological`, una de las menos representadas, ha pasado de que no se prediga bien **ninguna** muestra, a que se prediga correctamente un **70%** de las muestras. Esto es un claro síntoma de que la estrategia aplicada funciona para tratar con clases desbalanceadas.
- Por otro lado, la clase más abundante, `Meteorological`, ha pasado de predecirse correctamente un 73% de las veces a un 36%.
- A pesar del inconveniente anterior, en este nuevo modelo se aprecia claramente la diagonal principal de la matriz de confusión.

En función de las prioridades y del problema a tratar, se podría decir que el segundo modelo es mejor que el primero, o al revés. De nuevo, esto es una decisión que se ha de tomar en función de los objetivos. Por ejemplo, si nuestro objetivo fuera el de predecir correctamente la clase `Meteorological`, el segundo modelo sería peor que el primero.

Sin embargo, como deseamos que el modelo sea capaz de explicar todas las clases, se elegirá el segundo modelo.

Estudiamos ahora la importancia de las variables utilizadas:

In [ ]:
best_dtree.steps[-1][1].feature_importances_

Comparando este resultado con el anterior, hacemos las siguientes reflexiones:

- Las variables importantes en el anterior modelo, también lo son para este. Sin embargo, este modelo tiene en cuenta **más** variables que el modelo anterior.
- Probablemente, las variables del análisis anterior eran las más importantes para predecir la clase `Meteorological`, que era la más grande. Al tener en cuenta las clases más pequeñas, entran en juego más variables. Por ejemplo, en el modelo anterior, la variable `Duration` estaba fuertemente correlacionada con la clase, probablemente porque todos los desastres meteorológicos tienen una duración común. Sin embargo, al tener en cuenta las demás clases, la importancia de esta variable se reduce (aunque siga siendo la más importante), y se tienen en cuenta variables como `Total Affected`, `Year`, `Total Damages Corregida` o `Total Deaths` (en orden de mayor a menor importancia).

# Parte 2: Ensembles, clustering y reglas de asociación

## Ensembles

Ahora, intentaremos mejorar el resultado anterior mediante la utilización de ensembles. Concretamente, probaremos con un `RandomForestClassifier`:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('rf_clf', RandomForestClassifier(class_weight='balanced'))])
summary_clf(rf_clf, X, y)

A primera vista, podemos comprobar que el modelo `RandomForestClassifier` con los hiperparámetros por defecto supera al `DecisionTreeClassifier` con los parámetros ajustados, pasando de un F1 score de 0.34 a 0.41. Por tanto, es de esperar que, con un ajuste adecuado de hiperparámetros, el modelo de random forest suponga una mejoría todavía más notable respecto al árbol de decisión.

Además, es evidente que el modelo se sobreajusta (overfits) a los datos, ya que predice correctamente **todos** los datos de entrenamiento. Este es otro motivo por el cual se han de ajustar los hiperpámetros, para intentar regularizar el modelo y disminuir su error de generalización.

En la página de `sklearn` (https://scikit-learn.org/stable/modules/ensemble.html#random-forest-parameters) recomiendan una serie de pautas a la hora de ajustar los hiperparámetros:

- Los principales hiperparámetros a ajustar son `n_estimators` (número de estimadores) y `max_features` (tamaño máximo del subconjunto de variables a tener en cuenta)
- Para la regresión, se recomienda utilizar todas las variables (`max_features=None`), mientras que para clasificación se recomienda utilizar la raíz cuadrada del número total (`max_features='sqrt'`)

En el siguiente enlace (https://stackoverflow.com/questions/20463281/how-do-i-solve-overfitting-in-random-forest-of-python-sklearn) también se discute cómo reducir el overfitting. Concretamente, se recomienda ajustar los siguientes parámetros:

- `n_estimators`: En general, cuando mayor sea el número de árboles utilizado, menor será la probabilidad de overfitting. Es importante recalcar que el tiempo de entrenamiento aumenta considerablemente en función del número de estimadores.

- `max_features`: Reducir el número de características utilizado por cada árbol suele disminuir el overfitting. Aunque en la página de sklearn recomiendan tomar la raíz cuadrada del número total de características, se probará con otros valores.

- `max_depth`: Cuanto menor sea su valor, más simples serán los modelos, y por tanto será menos probable que haya overfitting.

- `min_samples_leaf`: Este parámetro indica el mínimo número de muestras que ha de tener un nodo para dividirse. Se probará a aumentar el valor de este parámetro.

In [ ]:
rf_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('rf_clf', RandomForestClassifier(class_weight='balanced',
                                                            n_estimators= 200,
                                                            max_features= 'sqrt',
                                                            max_depth= 40,
                                                            min_samples_leaf= 4))])
summary_clf(rf_clf, X, y)

Ajustando los parámetros se consigue aumentar el F1 de 0.43 a 0.45, aproximadamente. Respecto al modelo de `DecisionTreeClassifier` utilizado anteriormente, el uso del modelo de ensemble `RandomForestClassifier` supone una mejora de aproximadamente una décima en el F1 score.

El modelo de Random Forest pertenece a los llamados **Bagging** ensembles, en el que se entrenan una serie de modelos débiles a partir de un subconjunto de características en un dataset obtenido mediante bootstrap.

Ahora utilizaremos un modelo que pertenece a otro grupo de ensembles llamados **Boosting**. La idea detrás de los modelos de ensemble boosting es la de **actualizar los pesos de cada muestra en función de las predicciones de cada modelo débil**. En otras palabras, se darán mas peso a las muestras clasificadas incorrectamente por un modelo débil, de manera que el siguiente modelo intente predecir correctamente dichas clases.

Concretamente, utilizaremos un modelo **Adaboost**:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

adaboost_clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('ada_clf', AdaBoostClassifier())])
summary_clf(adaboost_clf, X, y)

Con los parámetros por defecto se obtiene un resultado incluso peor que con un modelo de regresión logística. Utilizaremos como estimador base un `DecisionTreeClassifier`, basándonos en los hiperparámetros que han funcionado correctamente para pruebas anteriores:

In [ ]:
adaboost_clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('ada_clf', AdaBoostClassifier(
                               n_estimators=200,
                               base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                    min_samples_split=7,
                                                                    min_samples_leaf=4,
                                                                    max_depth=10)))])
summary_clf(adaboost_clf, X, y)

Se consigue mejorar el resultado, pero se obtienen resultados similares a los obtenidos con un modelo más simple, como el de regresión logística.

Finalmente, probaremos un último modelo perteneciente al grupo de los **stacking** ensembles. La idea básica consiste en utilizar las predicciones de varios modelos como **entrada a un modelo final**, que tomará la decisión en función de estas predicciones.

Probaremos a utilizar como estimatores base un modelo de regresión logística y el árbol de decisión obtenidos anteriormente. Como estimador final, utilizaremos otro modelo de regresión logística:

In [ ]:
from sklearn.ensemble import StackingClassifier

estimators = [log.steps[-1], best_dtree.steps[-1]]

stacking_clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('stacking_clf', StackingClassifier(estimators,
                                                    final_estimator=LogisticRegression(max_iter=4000)))])
summary_clf(stacking_clf, X, y)

El ensemble formado por ambos modelos no parece mejorar el F1 score. Por último, probaremos a utilizar el modelo de regresión logística y el de random forest:

In [ ]:
estimators = [log.steps[-1], rf_clf.steps[-1]]

stacking_clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('stacking_clf', StackingClassifier(estimators,
                                                    final_estimator=LogisticRegression(max_iter=4000)))])
summary_clf(stacking_clf, X, y)

De nuevo, el ensemble funciona incluso peor que el modelo de random forest individual. 

Por tanto, podemos concluir que, de todos los modelos probados, el de random forest es el que mejor resultados proporciona. Resulta conveniente representar la matriz de confusión para analizar los resultados con más detalle:

In [ ]:
# hacemos un split en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Ajustamos el modelo
rf_clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('rf_clf', RandomForestClassifier(class_weight='balanced',
                                                            n_estimators= 200,
                                                            max_features= 'sqrt',
                                                            max_depth= 40,
                                                            min_samples_leaf= 4))])
rf_clf.fit(X_train, y_train)

# predecimos las clases
y_pred = rf_clf.predict(X_test)

# calculamos la matriz de confusión
plot_confusion_matrix(rf_clf, X_test, y_test, xticks_rotation=90, normalize='true')
plt.show()

Comparando con los resultados obtenidos con el `DecisionTreeClassifier`, se puede observar una mejoría considerable en las predicciones; sobretodo en la clasificación de la clase `Meteorological`: se ha pasado de predecir correctamente un 36% de las muestras a  un **58%**.

## Clustering

Para la parte de clustering, se intentará predecir a qué clase pertenece cada campeón de League of Legends en función de sus estadísticas. League of Legends es un juego multijugador competitivo, en el que típicamente se enfrentan dos grupos de 5 jugadores cada uno. Cada jugador elegirá uno de los 146 campeones (número de campeones en el parche 9.22). Cada campeón tiene un conjunto de habilidades y estadísticas que lo caracterizan.

Los campeones se pueden separar en 7 clases en función del rol al que mejor se adaptan (https://leagueoflegends.fandom.com/wiki/Champion_classes):

- **Controller** (Controlador): Los controladores asisten a sus aliados gracias a sus habilidades de utilidad. 

- **Fighter/Bruiser** (Luchador): Los luchadores son campeones caracterizados por tener un balance entre resistir daño e infligirlo.

- **Mage** (Mago): Los magos se caracterizan por infligir una gran cantidad de daño mágico, además de tener ciertas habilidades de control. Suelen tener vida reducida.

- **Marksman** (Francotirador): Caracterizados por infligir daño físico a distancia, suelen tener vida reducida.

- **Slayer** (Asesino): Suelen infligir una gran cantidad de daño a corto alcance y tener mucha movilidad, tienen vida reducida.

- **Tank** (Tanque): Caracterizados por su gran capacidad de resistir daño y sus efectos de control.

- **Especialista**: Esta clase engloba a todos los campeones que no encajan en las clases anteriores.

En (https://leagueoflegends.fandom.com/wiki/List_of_championshttps://leagueoflegends.fandom.com/wiki/List_of_champions) se puede consultar a qué clase pertenece cada campeón

In [ ]:
df_lol = pd.read_csv('../input/league-of-legends-champion-stats-922/champions-stats-09-22.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_lol)

El dataset contiene las estadísticas de cada uno de los 150 campeones que existen en el parche 10.02. A continuación se describen brevemente estas estadísticas:

- `name`: Nombre del campeón.
- `hp`: Puntos de vida iniciales.
- `hp+`: Estadística de crecimiento de la vida en función del nivel. Algunas estadísticas aumentan en función del nivel, siguiendo la fórmula $Statistic = b + g(n-1)(0.7025+0.0175(n-1))$, donde b sería el valor inicial (en el caso de la vida, `hp`) y g sería la estadística de crecimiento (en este caso, `hp+`).
- `hp5`/`hp5+`: Regeneración de vida cada 5 segundos
- `mp`/`mp+`: Puntos de maná.
- `mp5`/`mp5+`: Regeneración de maná cada 5 segundos.
- `AD`/`AD+`: Daño de ataque (físico)
- `AS`/`AS+`: Velocidad de ataque
- `AR`/`AR+`: Armadura (resistencia física)
- `MR`/`MR+`: Resistencia mágica
- `MS`: Velocidad de movimiento.
- `rng`: Rango de ataque.

A partir de estos stats, intentaremos agrupar los campeones en las 7 clases definidas. Para ello, aplicaremos el algoritmo de Mini Batch K-means, recomendado por https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html:

In [ ]:
from sklearn.cluster import MiniBatchKMeans

# Tomamos las estadísticas de cada campeón
X = df_lol.drop(columns={'name'})
# Estandarizamos las características para que tengan media cero y std 1
X = StandardScaler().fit_transform(X)
# Creamos el modelo K-means y ajustamos los datos, indicando que queremos 
# dividir los datos en 7 clusters (o clases)
kmeans = MiniBatchKMeans(n_clusters=7, random_state=42).fit(X)
# Obtenemos las etiquetas obtenidas
df_lol['Label'] = kmeans.labels_

Obtenemos la clasificación de los campeones:

In [ ]:
for label in np.unique(kmeans.labels_):
    champions = df_lol[df_lol['Label'] == label]['name']
    print("CLASE {}".format(label))
    print("Campeones: {}\n\n".format(champions.values))

En este primer intento de aplicar clustering, se pueden observar ciertas agrupaciones de campeones interesantes:

- La clase 1 parece representar a los **marksman**, por presencia de campeones como Ashe, Caitlyn, Draven, Jhin, Jinx, KaiSa, Kalista...
- De la misma manera, la clase 2 parece estar asociado a los **magos**.
- La clase 3 solamente contiene asesinos.

Sin embargo, la clase 5 es demasiado generalista, y engloba tanto a tanques, como magos, como luchadores...

Probablemente se podrían mejorar los resultados mediante la **eliminación de un cluster** asociado a la clase especialista, ya que esta clase está formada por campeones con stats muy diversos.

In [ ]:
# Tomamos las estadísticas de cada campeón
X = df_lol.drop(columns={'name'})
# Estandarizamos las características para que tengan media cero y std 1
X = StandardScaler().fit_transform(X)
# Creamos el modelo K-means y ajustamos los datos, indicando que queremos 
# dividir los datos en 7 clusters (o clases)
kmeans = MiniBatchKMeans(n_clusters=6, random_state=42).fit(X)
# Obtenemos las etiquetas obtenidas
df_lol['Label'] = kmeans.labels_
for label in np.unique(kmeans.labels_):
    champions = df_lol[df_lol['Label'] == label]['name']
    print("CLASE {}".format(label))
    print("Campeones: {}\n\n".format(champions.values))

Tras analizar los distintos clusters, podemos asociar las siguientes clases a cada cluster:

- Clase 0: Mages
- Clase 1: Fighters
- Clase 2: Controllers
- Clase 3: Marksmen
- Clase 4: Slayers
- Clase 5: Tanks

Sin embargo, el clustering no es perfecto, y hay campeones que se han agrupado incorrectamente. A continuación presentamos algunas observaciones:

- La clase de los **slayers** (asesinos) está formada completamente por asesinos (excepto Shen); el resto de asesinos ha sido agrupado incorrectamente en otras clases, como Katarina en la clase de los tanks, o fizz en la de los controllers.
- Parece que las diferencias entre la clase tank y fighter son muy leves, y hay cierta confusión a la hora de agrupar estas dos clases.
- Los clusters que mejor han agrupado son aquellos asociados a la clase **mage** y **marksman**. Este resultado era de esperar, ya que los campeones de estas clases tienen stats muy diferenciados respecto al resto: los magos se caracterizan por tener mucho maná y rango, y los marksman por tener mucho daño de ataque.

# Conclusiones

A lo largo de esta práctica, nos hemos familiarizado con el uso de modelos para los problemas de regresión y clasificación (aprendizaje supervisado) y clustering (aprendizaje no supervisado.

Además, ha servido como una primera toma de contacto con la aplicación de técnicas de aprendizaje automático en un dataset real. Es decir, mientras que en muchas ocasiones se utilizan datasets ya preparados con fines didácticos, aquí se ha utilizado un dataset que se ha tenido que preprocesar para poder aplicar los modelos.

Respecto a la parte de **regresión**, se han intentado predecir los daños causados por cada desastre. Sin embargo, los modelos aplicados proporcionaban un $R^2$ negativo o muy cercano a cero. Esto podría deberse por dos razones:

1. La variable que se intentaba predecir depende de otras variables que no se han tenido en cuenta.
2. El fenómeno es prácticamente aleatorio, y por tanto es un problema incompatible con técnicas de machine learning.

En este caso, es más probable que sea por el segundo motivo, ya que tanto los desastres como sus efectos suelen ser algo impredecible, y por tanto, el coste ocasionado también.

Respecto a la parte de **clasificación**, se intentó aplicar una serie de modelos para intentar encontrar la relación entre una serie de variables (duración del desastre, número de afectados...) con el tipo de desastre (Meteorológico, Tecnológico...). En este caso, si que se consiguió obtener un modelo (concretamente, un árbol de decisión) que conseguía captar dicha relación, obteniendo un F1 score de 0.34 en el conjunto de test. Se discutió también la elección de la métrica. 

También es importante recalcar que, para obtener dicho F1 score se tuvo que aplicar una técnica para tratar con datos balanceados, ya que habían clases con un número elevado de muestras respecto a las demás. Concretamente, se modificó el peso de cada muestra durante el entrenamiento en función del número de muestras por clase, mediante el argumento `class_weight='balanced'`. 

A continuación, se intentó mejorar el resultado de la parte de clasificación mediante modelos de **ensemble**. En concreto, el clasificador de **Random Forest** proporcionaba un F1 score de 0.45 en el conjunto de test.

Finalmente, para la parte de clustering se utilizó un dataset diferente, compuesto por las estadísticas de cada campeón del juego League of Legends, con el objetivo de agrupar los campeones en clases diferentes según su rol o características. Como el número de muestras era reducido, se aplicó el algoritmo de **mini batch k-means**. En un primer intento, se agruparon los campeones en 7 clases. Sin embargo, los resultados no fueron muy buenos, probablemente porque una de las clases (especialist) es difícil de agrupar en función de las estadísticas de cada campeón. Tras reducir el número de clusters a 6, se obtuvieron resultados considerablemente mejores, y se consiguieron diferenciar las diferentes clases con bastante claridad.
